# read in raw data, apply selection criteria, generate summary table and plot values

In [2]:
import os
import gdown
import zipfile

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import cartopy.crs as ccrs

from IPython.display import display
from PyOptimalInterpolation import get_data_path
from PyOptimalInterpolation.dataloader import DataLoader
from PyOptimalInterpolation.utils import config_func, stats_on_vals, WGS84toEASE2_New, EASE2toWGS84_New
from PyOptimalInterpolation.plot_utils import plot_pcolormesh, plot_hist

pd.set_option('display.max_columns', 200)

# read data - parameters / configuration

In [3]:
# ndf file to read from
hdf_file = get_data_path("RAW", "sats_ra_cry_processed_arco.h5")
# netCDF file to write to (for binned data)
ncdf_file = get_data_path("binned", "sats_ra_cry_processed_arco.nc")

table = "data"
val_col = "elev_mss"
lon_col = "lon"
lat_col = "lat"

scatter_plot_size = 2


# Read Data

In [4]:
print("reading from hdf5 files")
# read by specifying file path
df = DataLoader.read_hdf(table=table, path=hdf_file)

print("head of data:")
print(df.head(3))

# get config used to generate file

reading from hdf5 files
head of data:
         lon        lat     elev      mss  type  conf  elev_mss
0  44.981714 -60.893821 -17.4438 -17.4419     1     5   -0.0019
1  44.984416 -60.894198 -17.3047 -17.4324     1     5    0.1277
2  44.987118 -60.894575 -17.4319 -17.4229     1     5   -0.0090


# stats on data

In [5]:
print("*" * 20)
print("summary / stats table on metric (use for trimming)")

vals = df[val_col].values
stats_df = stats_on_vals(vals=vals, name=val_col,
                         qs=[0.01, 0.05] + np.arange(0.1, 1.0, 0.1).tolist() + [0.95, 0.99])

# print(stats_df)
display(stats_df)

********************
summary / stats table on metric (use for trimming)


,elev_mss
measure,NaN
size,2.503928e+07
num_not_nan,2.503928e+07
num_inf,0.000000e+00
min,-2.402095e+02
mean,3.746928e-01
max,1.778943e+03
std,1.454007e+01
skew,5.489539e+01
kurtosis,3.524301e+03


# select subset of data

In [6]:

plt_where = [
    {"col": val_col, "comp": ">=", "val": -1.0},
    {"col": val_col, "comp": "<=", "val": 1.0}
]

plt_df = DataLoader.data_select(df, where=plt_where)

plt_stats_df = stats_on_vals(vals=plt_df[val_col].values, name=val_col,
                             qs=[0.01, 0.05] + np.arange(0.1, 1.0, 0.1).tolist() + [0.95, 0.99])

display(plt_stats_df)


,elev_mss
measure,NaN
size,2.486041e+07
num_not_nan,2.486041e+07
num_inf,0.000000e+00
min,-1.000000e+00
mean,3.956344e-02
max,1.000000e+00
std,1.933668e-01
skew,-2.513607e-01
kurtosis,2.587982e+00


# plot (selected) values

In [7]:
df.head(2)

,lon,lat,elev,mss,type,conf,elev_mss
0,44.981714,-60.893821,-17.4438,-17.4419,1,5,-0.0019
1,44.984416,-60.894198,-17.3047,-17.4324,1,5,0.1277


In [12]:

figsize = (10, 5)
fig = plt.figure(figsize=figsize)

# figure title
where_print = ", ".join([" ".join([str(v) for k, v in pw.items()]) for pw in plt_where])
# put data source in here?
sup_title = f"val_col: {val_col}\n" \
            f"min_datetime {str(plt_df['datetime'].min())}, " \
            f"max datetime: {str(plt_df['datetime'].max())} \n" \
            f"where conditions:" + where_print
fig.suptitle(sup_title, fontsize=10)

nrows, ncols = 1, 2

# first plot: heat map of observations
ax = fig.add_subplot(1, 2, 1,
                     projection=ccrs.NorthPolarStereo())

plot_pcolormesh(ax=ax,
                lon=plt_df[lon_col].values,
                lat=plt_df[lat_col].values,
                plot_data=plt_df[val_col].values,
                fig=fig,
                # title=plt_title,
                # vmin=vmin,
                # vmax=vmax,
                cmap='YlGnBu_r',
                # cbar_label=cbar_labels[midx],
                scatter=True,
                s=scatter_plot_size)

ax = fig.add_subplot(1, 2, 2)

plot_hist(ax=ax,
          data=plt_df[val_col].values,
          ylabel="",
          stats_values=['mean', 'std', 'skew', 'kurtosis', 'min', 'max', 'num obs'],
          title=f"{val_col}",
          xlabel=val_col,
          stats_loc=(0.2, 0.8))

plt.tight_layout()
plt.show()


********************
visualise values
selecting rows (for plotting)
{'func': '<=', 'col_args': 'elev-mss', 'args': 5}
{'func': '>=', 'col_args': 'elev-mss', 'args': -5}
selecting 696294/740595 rows for plot


# Bin Data

In [ ]:


# convert 'datetime' to date
plt_df['date'] = plt_df['datetime'].values.astype('datetime64[D]')
plt_df['x'], plt_df['y'] = WGS84toEASE2_New(plt_df['lon'], plt_df['lat'])

# get a Dataset of binned data
ds_bin = DataLoader.bin_data_by(df=plt_df,
                                by_cols=['sat', 'date'],
                                val_col='elev_mss',
                                grid_res=50 * 1000,
                                x_range=[-4500000.0, 4500000.0],
                                y_range=[-4500000.0, 4500000.0])

# add lon,lat grid values to coords
x_grid, y_grid = np.meshgrid(ds_bin.coords['x'], ds_bin.coords['y'])
lon_grid, lat_grid = EASE2toWGS84_New(x_grid, y_grid)

ds_bin = ds_bin.assign_coords({"lon": (['y', 'x'], lon_grid),
                               "lat": (['y', 'x'], lat_grid)})

# write to file - mode = 'w' will overwrite file (?)
DataLoader.write_to_netcdf(ds=ds_bin, path=ncdf_file, mode="w")



# combine binned data into a single nd-array

# get lon, lat values for 2-d array

In [ ]:
# get the x,y values at the center of grid
src = list(bin_src.keys())[0]
x_edge, y_edge = bin_src[src][1], bin_src[src][2]

# get the centers for edges
x_cntr, y_cntr = x_edge[:-1] + np.diff(x_edge) / 2, y_edge[:-1] + np.diff(y_edge) / 2

# put in 2-d / grid form
x_grid, y_grid = np.meshgrid(x_cntr, y_cntr)

# convert to lon, lat
lon_grid, lat_grid = EASE2toWGS84_New(x_grid, y_grid)

# plot binned valued (averaged over time period)


In [ ]:

figsize = (10, 5)
fig = plt.figure(figsize=figsize)
fig.suptitle(sup_title)

nrows, ncols = 1, 2

# first plot: heat map of observations
ax = fig.add_subplot(nrows, ncols, 1,
                     projection=ccrs.NorthPolarStereo())

plot_pcolormesh(ax=ax,
                lon=lon_grid,
                lat=lat_grid,
                plot_data=ave_obs,
                fig=fig,
                title="(averaged) binned obs",
                vmin=vmin,
                vmax=vmax,
                cmap='YlGnBu_r',
                # cbar_label=cbar_labels[midx],
                scatter=False)

ax = fig.add_subplot(nrows, ncols, 2)

plot_hist(ax=ax,
          data=ave_obs[~np.isnan(ave_obs)],
          ylabel="",
          stats_values=['mean', 'std', 'skew', 'kurtosis', 'min', 'max', 'num obs'],
          title=f"elev - mss",
          xlabel="elev - mss (m)",
          stats_loc=(0.2, 0.85))

plt.tight_layout()
plt.show()
